In [2]:
import xgboost as xgb
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import csv

In [3]:
def show_accuracy(a, b, tip):
    acc = a.ravel() == b.ravel()
    acc_rate = 100 * float(acc.sum()) / a.size
    # print '%s正确率：%.3f%%' % (tip, acc_rate)
    return acc_rate

In [16]:
def load_data(file_name, is_train):
    data = pd.read_excel(file_name)  # 数据文件路径
    # print data.describe()
    
    
    x = data[['iyear', 'extended','country_txt', 'region', 'crit1', 'crit2', 'crit3', 'suicide', 'attacktype1', 'targtype1','weaptype1']]
    #x = data[['iyear', 'extended', 'crit1', 'crit2', 'crit3', 'suicide', 'region_1', 'region_2', 'region_3','region_4','region_5','region_6','region_7','region_8','region_9','region_10','region_11','region_12','attacktype1_1','attacktype1_2','attacktype1_3','attacktype1_4','attacktype1_5','attacktype1_6','attacktype1_7','attacktype1_8','attacktype1_9','targtype1_1','targtype1_2','targtype1_3','targtype1_4','targtype1_5','targtype1_6','targtype1_7','targtype1_8','targtype1_9','targtype1_10','targtype1_11','targtype1_12','targtype1_13','targtype1_14','targtype1_15','targtype1_16','targtype1_17','targtype1_18','targtype1_19','targtype1_20','targtype1_21','targtype1_22','weaptype1_1','weaptype1_2','weaptype1_3','weaptype1_5','weaptype1_6','weaptype1_7','weaptype1_8','weaptype1_9','weaptype1_10','weaptype1_11','weaptype1_12','weaptype1_13','country_txt_1','country_txt_2','country_txt_4']]
    # x = data[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']]
    y = None
    if 'gname1' in data:
        y = data['gname1']

    x = np.array(x)
    y = np.array(y)

    # 思考：这样做，其实发生了什么？
#     x = np.tile(x, (5, 1))
#     y = np.tile(y, (5, ))
    if is_train:
        return x, y
    return x, data['PassengerId']

In [17]:
x, y = load_data('question2_new_v2.xlsx', True)

In [18]:
print(x.shape)
print(y.shape)

(51609, 11)
(51609,)


In [20]:

#x, y = load_data('question2_new_v1.csv', True)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.7, random_state=1)


# XGBoost
data_train = xgb.DMatrix(x_train, label=y_train)
data_test = xgb.DMatrix(x_test, label=y_test)
watch_list = [(data_test, 'eval'), (data_train, 'train')]
# eta = learning_rate
param = {'max_depth': 2, 'eta': 0.3, 'silent': 1, 'objective': 'multi:softmax', 'num_class': 437}
#param = {'max_depth': 3, 'eta': 0.1, 'silent': 1, 'objective': 'binary:logistic'}
         # 'subsample': 1, 'alpha': 0, 'lambda': 0, 'min_child_weight': 1}
bst = xgb.train(param, data_train, num_boost_round=6, evals=watch_list)
y_hat = bst.predict(data_test)
# write_result(bst, 3)
# y_hat[y_hat > 0.5] = 1
# y_hat[~(y_hat > 0.5)] = 0
result = y_test.reshape(1, -1) == y_hat
#xgb_rate = show_accuracy(y_hat, y_test, 'XGBoost ')

print('正确率:\t', float(np.sum(result)) / len(y_hat))
#print('XGBoost：%.3f%%' % xgb_rate)

[0]	eval-merror:0.569463	train-merror:0.567756
[1]	eval-merror:0.96385	train-merror:0.963054


KeyboardInterrupt: 